[View in Colaboratory](https://colab.research.google.com/github/timohe/ML-Keras_ConvNeuralNet/blob/master/cifar10_cnn_exam_solution.ipynb)

# CIFAR10 with CNN
code from https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py


This notebook trains a simple convolutional neural network on the CIFAR10 small images dataset. 



In [0]:
# import libraries
from __future__ import print_function

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

#for confusion matrix
!pip install -q pandas_ml
import pandas_ml


Prepare data

In [9]:
# define constants
batch_size = 128
num_classes = 10
epochs = 2


# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Cast features into correct data type then scale features
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


Define model

In [0]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


Show model structure

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
__________

Compile model and fit


In [11]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])


model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)




Train on 50000 samples, validate on 10000 samples
Epoch 1/2
50000/50000 [==============================] - 258s 5ms/step - loss: 1.9720 - acc: 0.2783 - val_loss: 1.7261 - val_acc: 0.3880
Epoch 2/2
19456/50000 [==========>...................] - ETA: 2:30 - loss: 1.7402 - acc: 0.3698

50000/50000 [==============================] - 258s 5ms/step - loss: 1.6847 - acc: 0.3860 - val_loss: 1.5494 - val_acc: 0.4430


In [12]:
# Save the original model
model.save('savedFiles/cifra10_base.h5')

OSError: ignored

In [0]:
# Print base model loss and accuracy
print('\n  - Base case:')
model = keras.models.load_model("savedFiles/cifra10_base.h5")
scores = model.evaluate(x_test, y_test, verbose=0)
y_test_predict = model.predict(self, x_test)
print(pandas_ml.ConfusionMatrix(y_test, y_test_predict))
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - Base case:
	Test loss:	 0.8829086751937866
	Test accuracy:	 0.69


In [0]:
# 1) Augment the data by adding noise. Discuss results.

# Save models with gaussian noise with different Standard Deviation

for e in [0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]:
    model = keras.models.load_model("savedFiles/cifra10_base.h5")
    model.add(keras.layers.GaussianNoise(e))
    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                    metrics=['accuracy'])
    model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=epochs,
                validation_data=(x_test, y_test),
                shuffle=True)
    model.save('savedFiles/cifra10_n'+str(e)+'.h5')

In [0]:
# Load models and print results
for e in [0.08,0.07,0.06,0.05,0.04,0.03,0.02,0.01]:
    model = keras.models.load_model('savedFiles/cifra10_n'+str(e)+'.h5')
    print('\n  - Gaussian Noise with '+str(e)+' stdev:')
    scores = model.evaluate(x_test, y_test, verbose=0)
    print('\tTest loss:\t', scores[0])
    print('\tTest accuracy:\t', scores[1])
    
# Load and print of the best loss and accuracy obtained
model = keras.models.load_model("savedFiles/cifra10_n0.02.h5")
print('\n\nBest results:')
print('  - Gaussian Noise with 0.02 stdev:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - Gaussian Noise with 0.08 stdev:
	Test loss:	 0.830376790714264
	Test accuracy:	 0.7118

  - Gaussian Noise with 0.07 stdev:
	Test loss:	 0.8354920146942139
	Test accuracy:	 0.7119

  - Gaussian Noise with 0.06 stdev:
	Test loss:	 0.8408714697837829
	Test accuracy:	 0.7097

  - Gaussian Noise with 0.05 stdev:
	Test loss:	 0.7833409372329712
	Test accuracy:	 0.7291

  - Gaussian Noise with 0.04 stdev:
	Test loss:	 0.803309340763092
	Test accuracy:	 0.7226

  - Gaussian Noise with 0.03 stdev:
	Test loss:	 0.782099357509613
	Test accuracy:	 0.735

  - Gaussian Noise with 0.02 stdev:
	Test loss:	 0.7369885484695434
	Test accuracy:	 0.742

  - Gaussian Noise with 0.01 stdev:
	Test loss:	 0.830383843421936
	Test accuracy:	 0.7174


Best results:
  - Gaussian Noise with 0.02 stdev:
	Test loss:	 0.7369885484695434
	Test accuracy:	 0.742


In [0]:
# 2) Add layers to the network to see if can improve performance 
# - try two additional configurations - provide short discussin of what you did and why. 
# In addition, discuss results and compare and contrast confusion matrices from the three 
# different configurations (original and your two creations).

# First model: batch normalization
from keras.layers.normalization import BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('savedFiles/cifra10_batch.h5')


Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 296s 6ms/step - loss: 1.9490 - acc: 0.3041 - val_loss: 1.5028 - val_acc: 0.4599
Epoch 2/20
50000/50000 [==============================] - 302s 6ms/step - loss: 1.5574 - acc: 0.4348 - val_loss: 1.3246 - val_acc: 0.5235
Epoch 3/20
50000/50000 [==============================] - 302s 6ms/step - loss: 1.4228 - acc: 0.4863 - val_loss: 1.2613 - val_acc: 0.5494
Epoch 4/20
50000/50000 [==============================] - 302s 6ms/step - loss: 1.3397 - acc: 0.5174 - val_loss: 1.1878 - val_acc: 0.5791
Epoch 5/20
50000/50000 [==============================] - 302s 6ms/step - loss: 1.2701 - acc: 0.5476 - val_loss: 1.1172 - val_acc: 0.6039
Epoch 6/20
50000/50000 [==============================] - 302s 6ms/step - loss: 1.2189 - acc: 0.5671 - val_loss: 1.0932 - val_acc: 0.6082
Epoch 7/20
50000/50000 [==============================] - 301s 6ms/step - loss: 1.1627 - acc: 0.5845 - val_loss: 1.0218 - 

In [1]:

model = keras.models.load_model('savedFiles/cifra10_batch.h5')
print('\n  - Model with Batch Normalization:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


y_pred_test = model.predict(x_test, y_test, verbose=0)
print(pandas_ml.ConfusionMatrix(y_test, y_pred_test))

NameError: ignored

Exercises:
1. Augment the data by adding noise.  Discuss results.
2. Add layers to the network to see if can improve performance - try two additional configurations - provide short discussin of what you did and why.  In addition, discuss results and compare and contrast confusion matrices from the three different configurations (original and your two creations). 
3. Try another method for dealing with overfitting. Discuss results.

In [0]:
# 2) Add layers to the network to see if can improve performance 
# - try two additional configurations - provide short discussin of what you did and why. 
# In addition, discuss results and compare and contrast confusion matrices from the three 
# different configurations (original and your two creations).

# Second model: regularizer

from keras.layers.normalization import BatchNormalization
from keras import regularizers

weight_decay = 1e-4

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('savedFiles/cifra10_regL2.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 139s 3ms/step - loss: 2.0065 - acc: 0.2682 - val_loss: 1.7413 - val_acc: 0.3876
Epoch 2/20
50000/50000 [==============================] - 137s 3ms/step - loss: 1.7118 - acc: 0.3826 - val_loss: 1.5890 - val_acc: 0.4288
Epoch 3/20
50000/50000 [==============================] - 139s 3ms/step - loss: 1.5883 - acc: 0.4279 - val_loss: 1.4806 - val_acc: 0.4748
Epoch 4/20
50000/50000 [==============================] - 137s 3ms/step - loss: 1.4952 - acc: 0.4624 - val_loss: 1.4071 - val_acc: 0.5048
Epoch 5/20
50000/50000 [==============================] - 136s 3ms/step - loss: 1.4269 - acc: 0.4901 - val_loss: 1.3790 - val_acc: 0.5182
Epoch 6/20
50000/50000 [==============================] - 139s 3ms/step - loss: 1.3737 - acc: 0.5120 - val_loss: 1.2660 - val_acc: 0.5527
Epoch 7/20
50000/50000 [==============================] - 136s 3ms/step - loss: 1.3281 - acc: 0.5305 - val_loss: 1.2483 - 

In [0]:
model= keras.models.load_model('savedFiles/cifra10_regL2.h5')
print('\n  - regularizer L2:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - regularizer L2:
	Test loss:	 0.9596688842773438
	Test accuracy:	 0.6666


In [0]:
# 3) Try another method for dealing with overfitting. Discuss results.

from keras.layers.normalization import BatchNormalization
from keras import regularizers

weight_decay = 1e-4

model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(weight_decay)))

model.add(BatchNormalization())

model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)

model.save('savedFiles/cifra10_batch_regL2.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 264s 5ms/step - loss: 1.9654 - acc: 0.3062 - val_loss: 1.5571 - val_acc: 0.4412
Epoch 2/20
50000/50000 [==============================] - 255s 5ms/step - loss: 1.5817 - acc: 0.4248 - val_loss: 1.3868 - val_acc: 0.5038
Epoch 3/20
50000/50000 [==============================] - 256s 5ms/step - loss: 1.4555 - acc: 0.4754 - val_loss: 1.3421 - val_acc: 0.5254
Epoch 4/20
50000/50000 [==============================] - 255s 5ms/step - loss: 1.3685 - acc: 0.5142 - val_loss: 1.2107 - val_acc: 0.5723
Epoch 5/20
50000/50000 [==============================] - 256s 5ms/step - loss: 1.3007 - acc: 0.5375 - val_loss: 1.2377 - val_acc: 0.5710
Epoch 6/20
50000/50000 [==============================] - 257s 5ms/step - loss: 1.2406 - acc: 0.5586 - val_loss: 1.1629 - val_acc: 0.5944
Epoch 7/20
50000/50000 [==============================] - 258s 5ms/step - loss: 1.1893 - acc: 0.5786 - val_loss: 1.0853 - 

In [0]:
model.save('savedFiles/cifra10_batch_regL2.h5')
print('\n  - Model with batch normalization and regularizer:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - Model with batch normalization and regularizer:
	Test loss:	 0.782516943359375
	Test accuracy:	 0.731


In [0]:
# 3) Try another method for dealing with overfitting. Discuss results.

model = keras.models.load_model('savedFiles/cifra10_batch.h5')
model.add(keras.layers.GaussianNoise(0.01))
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
                metrics=['accuracy'])
model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            shuffle=True)
model.save('savedFiles/cifra10_batch_n0.01.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 264s 5ms/step - loss: 0.8254 - acc: 0.7253 - val_loss: 0.7493 - val_acc: 0.7401
Epoch 2/20
50000/50000 [==============================] - 257s 5ms/step - loss: 0.8272 - acc: 0.7266 - val_loss: 0.7915 - val_acc: 0.7283
Epoch 3/20
50000/50000 [==============================] - 256s 5ms/step - loss: 0.8099 - acc: 0.7335 - val_loss: 0.7576 - val_acc: 0.7412
Epoch 4/20
50000/50000 [==============================] - 256s 5ms/step - loss: 0.7952 - acc: 0.7338 - val_loss: 0.7867 - val_acc: 0.7304
Epoch 5/20
50000/50000 [==============================] - 257s 5ms/step - loss: 0.7850 - acc: 0.7397 - val_loss: 0.7258 - val_acc: 0.7470
Epoch 6/20
50000/50000 [==============================] - 260s 5ms/step - loss: 0.7818 - acc: 0.7436 - val_loss: 0.7408 - val_acc: 0.7446
Epoch 7/20
50000/50000 [==============================] - 296s 6ms/step - loss: 0.7811 - acc: 0.7459 - val_loss: 0.7435 - 

In [0]:
model = keras.models.load_model('savedFiles/cifra10_batch_n0.01.h5')

print('\n  - Model with Batch Normalization and Gaussian Noise with 0.01 stdev:')
scores = model.evaluate(x_test, y_test, verbose=0)
print('\tTest loss:\t', scores[0])
print('\tTest accuracy:\t', scores[1])


  - Model with Batch Normalization and Gaussian Noise with 0.01 stdev:
	Test loss:	 0.6423829383373261
	Test accuracy:	 0.776
